In [37]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import widgets
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

In [38]:
# Parameters

# dataset = 'infectious'
# beta = 1
# Delta_T = 0.1 # Fraction of dataset to select (this is then divided by phi)
# k = 10
# phi = 0.1
# lines = 0
# runs = 3

def get_data(dataset, Delta_T, phi):
    lines = 0
    if dataset == "infectious":
        lines = 17298
    elif dataset == "ht09_contact":
        lines = 20818
    elif dataset == "SFHH":
        lines = 70261
    elif dataset == "tij":
        lines = 78249

    total_edges = int(lines * Delta_T)
    random_start_point = np.random.uniform(0, float(1 - Delta_T))
    start = int(lines * random_start_point)

    train_lines = int(total_edges * float(1 - phi))
    mid = start + train_lines
    end = start + total_edges

    print(f'Total lines: {total_edges}, train lines {train_lines}')
    print(f'Starting at {start} till {mid} to predict till {end}')

    path_to_file = f'../data/{dataset}.txt'

    all_data = np.loadtxt(path_to_file, delimiter='\t', dtype=int)[start: end]
    train_data = all_data[:train_lines]
    predict_data = all_data[train_lines:]
    print(f'Starting at {train_data[0]} till {predict_data[0]} to predict till {predict_data[-1]}')


    return all_data, train_data, predict_data, train_lines, total_edges, start, mid, end

# Check all edges are unique

# a = np.array([src, dst, ts])
# a = a.T
# # np.any(np.all(np.unique(a, axis=1) == a, axis=1))
# u, c = np.unique(a, return_counts=True, axis=0)
# u[c > 1]

In [39]:
# Construct Graph
def make_graph(all_data):
    src, dst, ts = [], [], []
    graph = nx.MultiGraph()

    for line in all_data:
        src.append(line[0])
        dst.append(line[1])
        ts.append(line[2])

        if graph.nodes.get(line[0]) is None:
            graph.add_node(line[0], is_infected=False, infected_at=math.inf, influence=-1, influences=None, pred=None, polyfit_prediction=None)
        if graph.nodes.get(line[1]) is None:
            graph.add_node(line[1], is_infected=False, infected_at=math.inf, influence=-1, influences=None, pred=None, polyfit_prediction=None)

        graph.add_edge(line[0], line[1], key=line[2], timestamp=line[2])

    src, dst, ts = np.array(src), np.array(dst), np.array(ts) # ts is given sorted
#     print(graph.nodes.get(src[0]))

    return src, dst, ts, graph

In [40]:
def fill_holes_timestamps(start, mid, end, influences):
    xs, ys = [], []
    for i in range(start, end + 1):
        if i < mid:
            xs.append((i, influences.get(str(i), 0)))
        else:
            ys.append((i, influences.get(str(i), 0)))
            
    return xs, ys

In [41]:
def check_edge(edge, g, inf, ins, beta):
    # TODO: use beta
    if g.nodes.get(edge[0])['is_infected'] and edge[2] > g.nodes.get(edge[0])['infected_at'] and not g.nodes.get(edge[1])['is_infected'] and np.random.random() <= beta:
        inf += 1
        g.nodes.get(edge[1])['in_infected'] = True
        g.nodes.get(edge[1])['infected_at'] = edge[2]
        ins[str(edge[2])] = ins.get(str(edge[2]), 0) + 1
    
    return inf, ins, g

def find_influence_of_node(edges, node, at, beta):
    g = nx.Graph()
    influence, influences, pred = 0, {}, {}
    
    for edge in edges:
        if g.nodes.get(edge[0]) is None:
            g.add_node(edge[0], is_infected=False, infected_at=math.inf)
        if g.nodes.get(edge[1]) is None:
            g.add_node(edge[1], is_infected=False, infected_at=math.inf)

    g.nodes.get(node)['is_infected'] = True
    g.nodes.get(node)['infected_at'] = at

    # TODO: Check influence. I think it's getting unset somewhere because of how Python handles pointers.
    for i, edge in enumerate(edges):
        influence, influences, g = check_edge(edge, g, influence, influences, beta)
        new_edge = (edge[1], edge[0], edge[2])
        influence, influences, g = check_edge(new_edge, g, influence, influences, beta)
        
    return influence, influences

# TODO: Fix this
def find_influence_in_graph(graph, all_data, beta):
    for i, edge in enumerate(all_data):
        for e in [edge[0], edge[1]]:
            if graph.nodes.get(e)['influence'] == -1:
                z, zz = find_influence_of_node(all_data, e, edge[2], beta)
                graph.nodes.get(e)['influence'] = z
                graph.nodes.get(e)['influences'] = zz

    most_influential_nodes = sorted(list(graph.nodes.data("influence")), key=lambda x: x[1], reverse=True)[:5]
    print(f"most influentual nodes: {most_influential_nodes}")
    return most_influential_nodes

In [42]:
def polyfit(data, new_points, dimension):
    print(f"new_points: {new_points}")
    k, v = list(map(lambda x: int(x), data.keys())), list(data.values())
    print(f"k: {len(k)}")
    print(f"v: {len(v)}")
    
    fit = np.polyfit(k, v, 1) # The use of 1 signifies a linear fit.

    line = np.poly1d(fit)
    new_points = np.arange(new_points) + (k[-1] + 1)
    new_p = line(new_points)
    print(f'New points predicted are: {len(new_p)}')

    return new_p
#     return np.cumsum(line(new_points))

In [45]:
def plot(most_influential_nodes, graph):
    clr = ['r', 'g', 'b', 'y', 'm']

    for i, node in enumerate(most_influential_nodes):
        plt.plot(list(graph.nodes.get(node[0])['pred'].keys()), list(graph.nodes.get(node[0])['pred'].values()), 'o', color=clr[i])
        #plt.plot(list(graph.nodes.get(node[0])['polyfit_prediction'].keys()), list(graph.nodes.get(node[0])['polyfit_prediction'].values()), 'o')
#         print(graph.nodes.get(node[0])['pred'])
    # save the plot
    plot_name = 's.png'
    plt.savefig(plot_name)
    plt.show()

In [43]:
def simulate(dataset, delta_t, phi, beta):
    print(f'Running simulation for dataset {dataset} with delta_t {delta_t} and phi {phi} and beta {beta}')
    flag = True

    while flag:
        flag = False 
        all_data, train_data, predict_data, train_lines, total_edges, start, mid, end = get_data(dataset, delta_t, phi)
        src, dst, ts, graph = make_graph(all_data)
        most_influential_nodes = find_influence_in_graph(graph, all_data, beta) # 5 most influential nodes [(node id, influence)]
        
        for node in most_influential_nodes:
            thisnode = graph.nodes.get(node[0])
            if thisnode is not {}:
                print(f"node: {node[0]}")
                thisnode['influences'], thisnode['pred'] = fill_holes_timestamps(train_data[0][2], \
                                                                                 predict_data[0][2], \
                                                                                 predict_data[-1][2], \
                                                                                 thisnode['influences'])
                print(f"preds: {thisnode['pred']}\n\n")
                if len(thisnode['pred']) > 3:
                    thisnode['polyfit_prediction'] = polyfit(thisnode['influences'], new_points=end - mid, dimension=1)

        
        for node in most_influential_nodes:
            if not bool(graph.nodes.get(node[0])['pred']):
                flag = True

    # Find all the unique keys in most_influential_nodes
    keys_pred = set()
    keys_polyfit = set()
    for node in most_influential_nodes:
        keys_pred.update(graph.nodes.get(node[0])['pred'].keys())
        keys_polyfit.update(graph.nodes.get(node[0])['polyfit_prediction'].keys())
    
    # if a node is missing a key, add it and set it to 0
    for node in most_influential_nodes:
        for key in keys_pred:
            if key not in graph.nodes.get(node[0])['pred']:
                graph.nodes.get(node[0])['pred'][key] = 0
        for key in keys_polyfit:
            if key not in graph.nodes.get(node[0])['polyfit_prediction']:
                graph.nodes.get(node[0])['polyfit_prediction'][key] = 0
                
    # for each node sort keys
    for node in most_influential_nodes:
        graph.nodes.get(node[0])['pred'] = dict(sorted(graph.nodes.get(node[0])['pred'].items(), key=lambda item: item[0]))
        graph.nodes.get(node[0])['polyfit_prediction'] = dict(sorted(graph.nodes.get(node[0])['polyfit_prediction'].items(), key=lambda item: item[0]))

    plot(most_influential_nodes, graph)

In [44]:
simulate("infectious", 0.1, 0.1, 1)

Running simulation for dataset infectious with delta_t 0.1 and phi 0.1 and beta 1
Total lines: 1729, train lines 1556
Starting at 15565 till 17121 to predict till 17294
Starting at [ 356  357 1069] till [ 367  396 1346] to predict till [ 399  403 1422]
most influentual nodes: [(386, 204), (387, 198), (375, 144), (378, 142), (374, 139)]
node: 386
preds: [(1346, 1), (1347, 1), (1348, 1), (1349, 1), (1350, 1), (1351, 2), (1352, 1), (1353, 1), (1354, 0), (1355, 0), (1356, 0), (1357, 0), (1358, 1), (1359, 1), (1360, 1), (1361, 0), (1362, 0), (1363, 1), (1364, 1), (1365, 0), (1366, 1), (1367, 0), (1368, 0), (1369, 0), (1370, 0), (1371, 0), (1372, 0), (1373, 0), (1374, 0), (1375, 0), (1376, 0), (1377, 0), (1378, 0), (1379, 0), (1380, 0), (1381, 0), (1382, 0), (1383, 0), (1384, 0), (1385, 0), (1386, 0), (1387, 0), (1388, 0), (1389, 0), (1390, 0), (1391, 0), (1392, 0), (1393, 0), (1394, 0), (1395, 0), (1396, 0), (1397, 0), (1398, 0), (1399, 0), (1400, 0), (1401, 0), (1402, 0), (1403, 0), (1404,

AttributeError: 'list' object has no attribute 'keys'

In [ ]:
dataset_dropdown = widgets.Dropdown(options=['infectious', 'ht09_contact', 'SFHH', 'tij_InVS15'], description='Dataset', disabled=False, continuous_update=True)
phi_slider = widgets.FloatSlider(min=0.1, max=0.9, step=0.01, value=0.1, continuous_update=True)
delta_t_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.1, continuous_update=True)
beta_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.75, continuous_update=True)

w = widgets.interactive(simulate, dataset=dataset_dropdown.value, delta_t=delta_t_slider.value, phi=phi_slider.value, beta=beta_slider.value)
display(w)

